In [1]:
df = spark.read.format("parquet")\
    .option("header","true")\
    .option("inferschema","true")\
    .load("s3://combinedfiles/merge/combined.parquet/*")

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1647364833410_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
df.printSchema()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- FL_DATE: string (nullable = true)
 |-- OP_CARRIER: string (nullable = true)
 |-- OP_CARRIER_FL_NUM: long (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- CRS_DEP_TIME: long (nullable = true)
 |-- DEP_TIME: double (nullable = true)
 |-- DEP_DELAY: double (nullable = true)
 |-- TAXI_OUT: double (nullable = true)
 |-- WHEELS_OFF: double (nullable = true)
 |-- WHEELS_ON: double (nullable = true)
 |-- TAXI_IN: double (nullable = true)
 |-- CRS_ARR_TIME: long (nullable = true)
 |-- ARR_TIME: double (nullable = true)
 |-- ARR_DELAY: double (nullable = true)
 |-- CANCELLED: double (nullable = true)
 |-- CANCELLATION_CODE: string (nullable = true)
 |-- DIVERTED: double (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- ACTUAL_ELAPSED_TIME: double (nullable = true)
 |-- AIR_TIME: double (nullable = true)
 |-- DISTANCE: double (nullable = true)
 |-- CARRIER_DELAY: double (nullable = true)
 |-- WEATHER_DELAY: double (nul

Deleting unnecessary columns

In [3]:
df = df.drop('Unnamed: 27')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
df.select("CANCELLED").distinct().show(50,truncate=False)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+
|CANCELLED|
+---------+
|1.0      |
|0.0      |
+---------+

In [5]:
from pyspark.sql.functions import col, column
df=df.where(col("CANCELLED")==0)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df=df.drop(col("CANCELLED"))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
print("Number of Columns", len(df.columns))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Number of Columns 26

In [8]:
#Since We deleted all cancelled==1 row hence we dont have any values except NULL
df.select("CANCELLATION_CODE").distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1

In [9]:
df=df.drop(col("CANCELLATION_CODE"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
print("Number of Columns", len(df.columns))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Number of Columns 25

In [11]:
df=df.where(col("DIVERTED")==0)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
df=df.drop(col("DIVERTED"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
print("Number of Rows: ", df.count())
print("Number of Columns", len(df.columns))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Number of Rows:  60692142
Number of Columns 24

In [14]:
df = df.drop('OP_CARRIER_FL_NUM','CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY','DEP_TIME', 'ARR_TIME')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
from pyspark.sql.functions import col, column
df=df.withColumn("CRS_DEP_TIME",col("CRS_DEP_TIME")/600).withColumn("WHEELS_OFF",col("WHEELS_OFF")/600).withColumn("WHEELS_ON",col("WHEELS_ON")/600).withColumn("CRS_ARR_TIME",col("CRS_ARR_TIME")/600)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
import pyspark.sql.functions as F
df=df.withColumn("CRS_DEP_TIME",F.col("CRS_DEP_TIME").cast("int")).withColumn("WHEELS_OFF",F.col("WHEELS_OFF").cast("int")).withColumn("WHEELS_ON",F.col("WHEELS_ON").cast("int")).withColumn("CRS_ARR_TIME",F.col("CRS_ARR_TIME").cast("int"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
from pyspark.sql.functions import month
df = df.withColumn('MONTH',month(df.FL_DATE))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
from pyspark.sql.functions import dayofweek
df = df.withColumn('WEEKDAY',dayofweek(df.FL_DATE))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
df=df.withColumn("FL_DATE",F.col("FL_DATE").cast("string"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
from pyspark.sql.functions import count,when,isnan

df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]
   ).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+------+----+------------+---------+--------+----------+---------+-------+------------+---------+----------------+-------------------+--------+--------+-----+-------+
|FL_DATE|OP_CARRIER|ORIGIN|DEST|CRS_DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_DELAY|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|MONTH|WEEKDAY|
+-------+----------+------+----+------------+---------+--------+----------+---------+-------+------------+---------+----------------+-------------------+--------+--------+-----+-------+
|      0|         0|     0|   0|           0|     4725|       0|         0|        2|      1|           0|     2600|               1|                  3|       2|       0|    0|      0|
+-------+----------+------+----+------------+---------+--------+----------+---------+-------+------------+---------+----------------+-------------------+--------+--------+-----+-------+

In [21]:
df = df.na.fill(value=0,subset=["DEP_DELAY"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
df = df.na.drop()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]
   ).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+------+----+------------+---------+--------+----------+---------+-------+------------+---------+----------------+-------------------+--------+--------+-----+-------+
|FL_DATE|OP_CARRIER|ORIGIN|DEST|CRS_DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_DELAY|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|MONTH|WEEKDAY|
+-------+----------+------+----+------------+---------+--------+----------+---------+-------+------------+---------+----------------+-------------------+--------+--------+-----+-------+
|      0|         0|     0|   0|           0|        0|       0|         0|        0|      0|           0|        0|               0|                  0|       0|       0|    0|      0|
+-------+----------+------+----+------------+---------+--------+----------+---------+-------+------------+---------+----------------+-------------------+--------+--------+-----+-------+

In [24]:
df.select("OP_CARRIER").distinct().show(50,truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+
|OP_CARRIER|
+----------+
|AS        |
|AA        |
|B6        |
|YV        |
|US        |
|MQ        |
|9E        |
|EV        |
|OO        |
|FL        |
|WN        |
|NW        |
|F9        |
|HA        |
|XE        |
|YX        |
|CO        |
|VX        |
|UA        |
|NK        |
|DL        |
|OH        |
|G4        |
+----------+

In [25]:
carrier_name = {'UA':'United Airlines',
    'AS':'Alaska Airlines',
    '9E':'Endeavor Air',
    'B6':'JetBlue Airways',
    'EV':'ExpressJet',
    'F9':'Frontier Airlines',
    'HA':'Hawaiian Airlines',
    'MQ':'Envoy Air',
    'NK':'Spirit Airlines',
    'OO':'SkyWest Airlines',
    'VX':'Virgin America',
    'WN':'Southwest Airlines',
    'YV':'Mesa Airline',
    'YX':'Republic Airways',
    'AA':'American Airlines',
    'US':'US Airways',
    'FL':'AirTran Airways Corporation',
    'NW':'Northwest Airlines',
    'CO':'Continental Air Lines',
    'XE':'Expressjet Airlines',
    'DL':'Delta Airlines',
    'OH':'Comair Airlines',
    'G4': 'Allegiant Airlines'}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
df = df.na.replace(carrier_name, 1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py:1803: UserWarning: to_replace is a dict and value is not None. value will be ignored.

In [27]:
df.groupBy('OP_CARRIER').count().show(50,truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------+--------+
|OP_CARRIER                 |count   |
+---------------------------+--------+
|United Airlines            |4733770 |
|Continental Air Lines      |732372  |
|Republic Airways           |305251  |
|Comair Airlines            |552941  |
|Mesa Airline               |1043935 |
|Virgin America             |386960  |
|Expressjet Airlines        |1074922 |
|ExpressJet                 |4577296 |
|SkyWest Airlines           |6136768 |
|Frontier Airlines          |890983  |
|JetBlue Airways            |2446659 |
|Northwest Airlines         |290097  |
|Allegiant Airlines         |95192   |
|Endeavor Air               |1324076 |
|American Airlines          |6566001 |
|Delta Airlines             |7792523 |
|Alaska Airlines            |1653104 |
|Spirit Airlines            |576161  |
|Hawaiian Airlines          |741090  |
|AirTran Airways Corporation|1159982 |
|Envoy Air                  |3055874 |
|US Airways                 |2624103 |
|Southwest Airlines      

In [28]:
df = df.withColumn("FLIGHT_STATUS", when(df.ARR_DELAY < 0 ,0)
                                    .otherwise(1))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
df.select("*").show(5,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------------+------+----+------------+---------+--------+----------+---------+-------+------------+---------+----------------+-------------------+--------+--------+-----+-------+-------------+
|FL_DATE   |OP_CARRIER         |ORIGIN|DEST|CRS_DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_DELAY|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|MONTH|WEEKDAY|FLIGHT_STATUS|
+----------+-------------------+------+----+------------+---------+--------+----------+---------+-------+------------+---------+----------------+-------------------+--------+--------+-----+-------+-------------+
|2009-01-01|Expressjet Airlines|DCA   |EWR |1           |-2.0     |18.0    |1         |1        |8.0    |2           |4.0      |62.0            |68.0               |42.0    |199.0   |1    |5      |1            |
|2009-01-01|Expressjet Airlines|EWR   |IAD |2           |-1.0     |28.0    |2         |2        |4.0    |2           |-8.0     |82.0            |75.0   

In [30]:
print("Number of Rows: ", df.count())
print("Number of Columns", len(df.columns))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Number of Rows:  60689539
Number of Columns 19

In [31]:
df.select("ORIGIN").distinct().count()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

378

In [32]:
df.select("DEST").distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

377

In [43]:
df.write.option("header","true").parquet("s3://combinedfiles/Final_Preprocessed/Preprocessed.parquet")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
from pyspark.ml.stat import Correlation

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
import six
for i in df.columns:
    if not( isinstance(df.select(i).take(1)[0][0], six.string_types)):
        print( "Correlation of Total Compensation with ", i, df.stat.corr('FLIGHT_STATUS',i))